# **Building a API with Path Parameters, Enums, and File Handling**


## Table of Contents
1. [Introduction to FastAPI](#introduction-to-fastapi)
2. [Path Parameters](#path-parameters)
   - [Basic Path Parameter Example](#basic-path-parameter-example)
   - [Type Annotated Path Parameters](#type-annotated-path-parameters)
3. [Route Ordering](#route-ordering)
4. [Using Enums in Routes](#using-enums-in-routes)
5. [File Handling in FastAPI](#file-handling-in-fastapi)
6. [Conclusion](#conclusion)

## **Introduction to FastAPI**

**FastAPI** is a Python framework designed for building APIs quickly and efficiently. It leverages Python's type hinting system to provide automatic data validation, serialization, and documentation. This makes it easy to define complex APIs with minimal boilerplate code.

To start, ensure you have FastAPI installed:
```bash
pip install fastapi uvicorn
```

We'll use `uvicorn` to run the application.

## **Path Parameters**

### 1. Basic Path Parameter Example

Path parameters allow you to extract values from the URL path. For example, if you want to retrieve information about a specific product, you can define a route like this:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/products/{product_id}")
async def read_product(product_id):
    return {"product_id": product_id}
```

#### Explanation:
- The route `/products/{product_id}` defines a dynamic segment `{product_id}`.
- When a GET request is made to `/products/123`, the value `123` is passed to the `read_product` function as the `product_id` parameter.
- The function returns a JSON response containing the `product_id`.

### 2. Type Annotated Path Parameters

You can enforce type checking on path parameters by specifying their types. For instance, if `product_id` should always be an integer:

```python
@app.get("/products/{product_id}")
async def read_product(product_id: int):
    return {"product_id": product_id}
```

#### Explanation:
- By annotating `product_id` as `int`, FastAPI ensures that only valid integers are accepted.
- If a non-integer value is provided (e.g., `/products/abc`), FastAPI automatically returns a 422 Unprocessable Entity error.

## **Route Ordering**

Route ordering is crucial when defining overlapping paths. Consider the following two routes:

```python
@app.get("/users/me")
async def read_current_user():
    return {"user_id": "the data for the current user"}

@app.get("/users/{user_id}")
async def read_user(user_id: str):
    return {"user_id": user_id}
```

#### Explanation:
- The `/users/me` route must come **before** `/users/{user_id}` because FastAPI matches routes in the order they are defined.
- If `/users/{user_id}` were defined first, the word `me` would be interpreted as a `user_id`, causing incorrect behavior.

## **Using Enums in Routes**

Enums (Enumerations) are useful for representing a fixed set of possible values. In FastAPI, you can use them to restrict input to predefined options.

### Defining an Enum

```python
from enum import Enum

class DeviceType(str, Enum):
    smartphone = "smartphone"
    tablet = "tablet"
    laptop = "laptop"
```

#### Explanation:
- The `DeviceType` class inherits from both `str` and `Enum`. This allows it to be serialized as a string in JSON responses.
- It defines three possible values: `smartphone`, `tablet`, and `laptop`.

### Using an Enum in a Route

```python
@app.get("/devices/{device_type}")
async def get_device(device_type: DeviceType):
    return {"device_type": device_type}
```

#### Explanation:
- The `device_type` parameter is annotated with `DeviceType`.
- Only values from the `DeviceType` enum (`smartphone`, `tablet`, or `laptop`) are accepted.
- If an invalid value is provided, FastAPI raises a 422 error.

## **File Handling in FastAPI**

FastAPI can handle file operations, such as reading files from the server's filesystem. Here's an example of serving file content:

```python
from pathlib import Path
from fastapi import HTTPException

@app.get("/docs/{file_path:path}")
async def read_document(file_path: str):
    file_location = Path("docs") / file_path
    
    if not file_location.exists() or not file_location.is_file():
        raise HTTPException(status_code=404, detail="file not found")
    
    with open(file_location, "r") as file:
        content = file.read()
    
    return {"content": content}
```

#### Explanation:
1. **Path Parameter with Colon Notation**:
   - The `file_path:path` syntax allows the parameter to capture the entire remaining path, including slashes.
2. **File Validation**:
   - The `Path` object checks if the file exists and is a regular file.
   - If the file is missing or invalid, an `HTTPException` with a 404 status code is raised.
3. **Reading File Content**:
   - The file is opened in read mode (`"r"`), and its content is returned as part of the JSON response.

## **Conclusion**

FastAPI simplifies the process of building robust APIs by providing powerful tools and conventions. With its strong support for type hints and automatic documentation, it's an excellent choice for modern web applications.